In [199]:
import pickle
import numpy as np
from decision_making import *
from decision_making.src.messages.scene_static_message import LaneSegmentConnectivity
from decision_making.src.messages.scene_static_enums import ManeuverType

In [200]:
scene = pickle.load(open("resources/scene_static_mocks/accel_scene_static.pkl", "rb"))

In [201]:
lanes  = scene.s_Data.s_SceneStaticBase.as_scene_lane_segments
roads = scene.s_Data.s_SceneStaticBase.as_scene_road_segment

def get_lane(lane_id):
    return [lane for lane in lanes if lane.e_i_lane_segment_id == lane_id][0]

def get_road(road_id):
    return [road_seg for road_seg in roads if road_seg.e_i_road_segment_id == road_id][0]


In [202]:
first_road_segment = get_road(46613)

In [203]:
curr_road = first_road_segment
#while(curr_road_seg.e_Cnt_downstream_segment_count > 0):

while curr_road.e_Cnt_downstream_segment_count > 0:
    #print(curr_road.e_i_road_segment_id)
    
    curr_num_lanes = curr_road.e_Cnt_lane_segment_id_count
    curr_lanes = [get_lane(lane_id) for lane_id in curr_road.a_i_lane_segment_ids]
    downstream_road = get_road(curr_road.a_i_downstream_road_segment_ids[0])
    
    print(str(curr_road.e_i_road_segment_id) + ',' + str(curr_num_lanes) + ',' + str(downstream_road.e_i_road_segment_id) + ',' + str(downstream_road.e_Cnt_lane_segment_id_count))
    # split case
    if downstream_road.e_Cnt_lane_segment_id_count > curr_num_lanes:
        downstream_lanes = [get_lane(lane_id) for lane_id in downstream_road.a_i_lane_segment_ids]
        
        # find lane with no upstream
        new_lane = [downstream_lane for downstream_lane in downstream_lanes if downstream_lane.e_Cnt_upstream_lane_count == 0]
       
        if len(new_lane) > 0:
            new_lane = new_lane [0]
        else:
            curr_road = downstream_road
            continue
    
        # make new connections depending on what side the lane is on
        if new_lane.e_Cnt_left_adjacent_lane_count == 0: # new lane is leftmost
            # get current leftmost lane
            leftmost_curr_lane = [ln for ln in curr_lanes if ln.e_Cnt_left_adjacent_lane_count == 0][0]
            # add downstream and upstream lane counts
            leftmost_curr_lane.e_Cnt_downstream_lane_count += 1
            new_lane.e_Cnt_upstream_lane_count += 1

            # add downstream and upstream lane id's 
            leftmost_curr_lane.as_downstream_lanes.append(LaneSegmentConnectivity(new_lane.e_i_lane_segment_id, ManeuverType.LEFT_SPLIT))
            new_lane.as_upstream_lanes.append(LaneSegmentConnectivity(leftmost_curr_lane.e_i_lane_segment_id, ManeuverType.LEFT_SPLIT))


        elif new_lane.e_Cnt_right_adjacent_lane_count == 0: # new lane is rightmost
            rightmost_curr_lane = [ln for ln in curr_lanes if ln.e_Cnt_right_adjacent_lane_count == 0][0]

            rightmost_curr_lane.e_Cnt_downstream_lane_count += 1 
            new_lane.e_Cnt_upstream_lane_count += 1

            rightmost_curr_lane.as_downstream_lanes.append(LaneSegmentConnectivity(new_lane.e_i_lane_segment_id, ManeuverType.RIGHT_SPLIT))
            new_lane.as_upstream_lanes.append(LaneSegmentConnectivity(rightmost_curr_lane.e_i_lane_segment_id, ManeuverType.RIGHT_SPLIT))

    # merge case
    if downstream_road.e_Cnt_lane_segment_id_count < curr_num_lanes:
        downstream_lanes = [get_lane(lane_id) for lane_id in downstream_road.a_i_lane_segment_ids]

        # find lane with no downstream
        disappearing_lane = [ln for ln in curr_lanes if ln.e_Cnt_downstream_lane_count == 0]
        if len(disappearing_lane) > 0:
            disappearing_lane = disappearing_lane[0]
        else:
            curr_road = downstream_road
            continue

        # make new connections depending on what side the lane is on
        if disappearing_lane.e_Cnt_left_adjacent_lane_count == 0: # merging lane is on the left
            # get current leftmost downstream lane
            leftmost_downstream_lane = [ln for ln in downstream_lanes if ln.e_Cnt_left_adjacent_lane_count == 0][0]

            # update upstream and downstream lane counts
            leftmost_downstream_lane.e_Cnt_upstream_lane_count += 1
            disappearing_lane.e_Cnt_downstream_lane_count += 1

            # update downstream and upstream lane id's 
            leftmost_downstream_lane.as_upstream_lanes.append(LaneSegmentConnectivity(disappearing_lane.e_i_lane_segment_id, ManeuverType.LEFT_MERGE_CONNECTION))
            disappearing_lane.as_downstream_lanes.append(LaneSegmentConnectivity(leftmost_downstream_lane.e_i_lane_segment_id ,ManeuverType.LEFT_MERGE_CONNECTION))

        elif disappearing_lane.e_Cnt_right_adjacent_lane_count == 0:
            rightmost_downstream_lane = [ln for ln in downstream_lanes if ln.e_Cnt_right_adjacent_lane_count == 0][0]

            # update upstream and downstream lane counts
            rightmost_downstream_lane.e_Cnt_upstream_lane_count += 1
            disappearing_lane.e_Cnt_downstream_lane_count += 1

            # update downstream and upstream lane id's
            rightmost_downstream_lane.as_upstream_lanes.append(LaneSegmentConnectivity(disappearing_lane.e_i_lane_segment_id, ManeuverType.RIGHT_MERGE_CONNECTION))
            disappearing_lane.as_downstream_lanes.append(LaneSegmentConnectivity(rightmost_downstream_lane.e_i_lane_segment_id, ManeuverType.RIGHT_MERGE_CONNECTION))
    # go on to next road segment
    curr_road = downstream_road

    

46613,4,87759,5
87759,5,8766,4
8766,4,76838,5
76838,5,228030,5
228030,5,51360,4
51360,4,228028,4
228028,4,87622,3
87622,3,228007,3
228007,3,87660,3
87660,3,87744,4
87744,4,9893,4
9893,4,9894,4
9894,4,87740,5
87740,5,77398,5
77398,5,87741,5
87741,5,25969,4
25969,4,10068,4
10068,4,87211,5
87211,5,10320,5
10320,5,10322,5
10322,5,228029,5
228029,5,87739,5
87739,5,40953,5
40953,5,10073,4
10073,4,10066,4
10066,4,87732,4
87732,4,43516,4
43516,4,87770,4
87770,4,228034,3
228034,3,87996,3
87996,3,228037,4
228037,4,10536,4
10536,4,88088,4
88088,4,228039,3
228039,3,88192,4
88192,4,10519,4


In [204]:
pickle.dump(scene, open("oval_with_splits.pkl", "wb"))

[<decision_making.src.messages.scene_static_message.SceneLaneSegmentBase object at 0x7f37555e4f28>, <decision_making.src.messages.scene_static_message.SceneLaneSegmentBase object at 0x7f37555e42e8>, <decision_making.src.messages.scene_static_message.SceneLaneSegmentBase object at 0x7f37555e4a90>, <decision_making.src.messages.scene_static_message.SceneLaneSegmentBase object at 0x7f37555e4be0>]


[0, 1]
